
# Cleaning & Geolocating

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


##  Load data

In [4]:
df = pd.read_csv('data/input/2023.csv')
pd.set_option('display.max_colwidth', None)
df.sample(5)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments
45922,2023-11-30 13:01:00,EL402,949120,ELE: EL402 - 223 - Lexington Av/63 St - Upper Plat,NaN,FR,Cancelled,Controller Power Loss,UNDERINVESTG,Under Investigation,2023-11-30 22:00:00,NaN,ST231130.txt - 1973,External Monitoring System,EE-LNOUT,2023-11-30 13:25:00,C,As per Galaxy Client elevator running
21694,2023-06-07 06:59:00,ES309,880096,ESC: ES309 - 026 - Dekalb Av - Manhattan Bound Plat,NaN,BRP,Closed,Not Running,UNDERINVESTG,Under Investigation,2023-06-07 15:00:00,2023-06-07 09:00:00,ST230607.txt - 1038,External Monitoring System,EE-LNOUT,2023-06-07 08:01:00,CL,NaN
32490,2023-08-22 12:49:00,ES210,910129,ESC: ES210 - 402 - Grand Central-42 St - Mezz A,NaN,BRP,Closed,Not Running,REPAIR,Repair,2023-08-22 21:00:00,2023-08-22 13:50:00,ST230822.txt - 2019,External Monitoring System,EE-LNOUT,2023-08-22 13:08:00,CL,NaN
17426,2023-05-08 19:10:00,ES208,849985,ESC: ES208 - 402 - Grand Central-42 St - Mezz A,NaN,TAN,Closed,Ready,REPAIR,Repair,2023-05-09 04:00:00,2023-05-08 20:35:00,ST230508.txt - 2873,External Monitoring System,EE-LNOUT,2023-05-08 19:20:00,CL,NaN
31719,2023-08-16 07:40:00,ES442,908004,ESC: ES442 - 278 - Jamaica Center-Parsons/Archer - Island Lower Plat,NaN,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-08-16 15:00:00,2023-08-16 12:45:00,NaN,Phone,EE-MANOUT,2023-08-16 07:43:00,CL,NaN


In [5]:
%%R
df <- read_csv('data/input/2023.csv')

Rows: 50296 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): Common Name, Equipment Description, Executive Comment, Outage Cod...
dbl   (1): Outage
dttm  (4): Out of Service Date, Estimated Return to Service Date, Actual Ret...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Clean data

In [6]:
# filter down to elevators only
df_ele = df[df['Equipment Description'].str.startswith('ELE')]

In [7]:
# create column `station MRN` to later merge with `Station ID` in another file to geocode
df_ele['Station MRN'] = df_ele['Equipment Description'].str.extract(r'ELE: EL\d{2,3}\w? - (\d{2,3})\s-')

In [8]:
# check what the are the missing station MRN
# missing values are all elevators that are either located in New Utrecht Av (MRN: 73) or Dyckman St Quarters(MRN: TBD)

df_ele[df_ele['Station MRN'].isna()].to_csv('data/missing-value/2023_null_station_mrn.csv', index=False)
df_ele = df_ele.dropna(subset=['Station MRN'])

In [9]:
# convert string to float
df_ele['Station MRN'] = df_ele['Station MRN'].astype('Int64')

# stations with MRN > 523 are not regular subway stations
# filter down to subway elevators 
df_subway_ele = df_ele[df_ele['Station MRN'] <= 523]
print(df_subway_ele.shape)

# save to a new file
df_subway_ele.to_csv('data/intermediary/2023_ele.csv', index=False)
df_ele[df_ele['Station MRN'] > 523].to_csv('data/missing-value/2023_ele_not_subway.csv', index=False)

(18051, 19)


## Geocoding: Adding lat long to dataset

Merging datasets by `Station ID`(subway_station.csv) and `Station MRN` (my dataset)

In [10]:

# load the geostation file
df_geostation = pd.read_csv('data/input/subway_stations.csv')

# merge the geostation file with the df_ele
df_ele_merged = df_subway_ele.merge(df_geostation, left_on='Station MRN', right_on='Station ID', how='inner')
df_ele_merged = df_ele_merged.drop_duplicates(subset=['Outage'])

# rename the columns
df_ele_merged = df_ele_merged.rename(columns={'GTFS Latitude': 'lat',  'GTFS Longitude': 'long'})
df_ele_merged.to_csv('data/intermediary/2023_ele_geocode.csv', index=False)


## Convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [11]:
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df_ele_merged['lat']
        longitudes = df_ele_merged['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df_ele_merged)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df_ele_merged['lat'], df_ele_merged['long']) 
census_geos_df.head()

  0%|          | 0/18051 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,610,360470296002002,+40.6008553,2002,0,36,2002,210701004650915,BK,...,029600,-073.9943207,2,20213,242,-073.9943207,G5040,L,U,047
1,,0,360050403021003,+40.8669415,1003,0,36,1003,210701006017414,BK,...,040302,-073.8935996,1,3799,0,-073.8935996,G5040,L,U,005
2,,593,360610034004000,+40.7305461,4000,0,36,4000,210701008621623,BK,...,003400,-073.9817046,4,19330,415,-073.9817046,G5040,L,U,061
3,,0,360810208001000,+40.6998869,1000,0,36,1000,210701006104178,BK,...,020800,-073.8089158,1,13876,0,-073.8089158,G5040,L,U,081
4,,0,360810033011019,+40.7492525,1019,0,36,1019,210701006117049,BK,...,003301,-073.9374018,1,3969,0,-073.9374018,G5040,L,U,081


In [12]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360470296002002,36,047,029600,2002
1,360050403021003,36,005,040302,1003
2,360610034004000,36,061,003400,4000
3,360810208001000,36,081,020800,1000
4,360810033011019,36,081,003301,1019
...,...,...,...,...,...
18046,360610137003000,36,061,013700,3000
18047,360610137003000,36,061,013700,3000
18048,360610137003000,36,061,013700,3000
18049,360050224031001,36,005,022403,1001


## Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [13]:
df_with_geos = pd.concat(
    [ 
        df_ele_merged.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

print(df_with_geos.shape)
df_with_geos.head()


(18051, 40)


,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,...,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-01-01 00:14:00,EL376,791238,ELE: EL376 - 068 - Bay Pkwy - Outside Area,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,...,Manhattan,Coney Island,1,NaN,POINT (-73.993728 40.601875),360470296002002,36,047,029600,2002
1,2023-01-01 01:04:00,EL189,791257,ELE: EL189 - 212 - Kingsbridge Rd - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,...,Bedford Pk Blvd & 205 St,Manhattan,1,NaN,POINT (-73.893509 40.866978),360050403021003,36,005,040302,1003
2,2023-01-01 02:01:00,EL293,791274,ELE: EL293 - 119 - 1 Av - Brooklyn Bound Platform,NaN,EUF,Closed,UPS Battery Failure,REPAIR,Repair,...,8 Av,Brooklyn,1,NaN,POINT (-73.981628 40.730953),360610034004000,36,061,003400,4000
3,2023-01-01 02:08:00,EL449X,791275,ELE: EL449X - 279 - Sutphin Blvd-Archer Av - Mezz,NaN,TPE,Closed,NaN,REPAIR,Repair,...,Jamaica Center,Manhattan,1,NaN,POINT (-73.807969 40.700486),360810208001000,36,081,020800,1000
4,2023-01-01 05:43:00,EL428,791299,ELE: EL428 - 273 - Queens Plaza - Outside Area,NaN,VAN,Closed,NaN,REPAIR,Repair,...,Forest Hills - Jamaica,Manhattan,1,NaN,POINT (-73.937243 40.748973),360810033011019,36,081,003301,1019


In [14]:
df_with_geos.to_csv('data/intermediary/2023_subway_censusgeo.csv', index=False)